In [1]:
from habitat.tasks.nav.nav import (
    NavigationEpisode,
    NavigationGoal,
    NavigationTask,
)

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [3]:
dir(NavigationGoal)

['__annotations__',
 '__attrs_attrs__',
 '__attrs_props__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

In [8]:
import gzip
import json

# ===== 修改为你的 train.json.gz 路径 =====
path = "/home/Disk/sound-space/data/datasets/semantic_audionav/mp3d/v1/test/content/split_2.json.gz"

with gzip.open(path, "rt", encoding="utf-8") as f:
    text = f.read()

print("File loaded.")
print("Text length:", len(text))


File loaded.
Text length: 70650


In [9]:
text

'{"episodes": [{"episode_id": "2845", "scene_id": "yqstnuAEVhm/yqstnuAEVhm.glb", "start_position": [0.632884, -0.004817000000000071, -10.618986], "start_rotation": [0.0, 0.7071067811865475, 0.0, 0.7071067811865476], "info": {"geodesic_distance": 4.0, "num_action": 7}, "goals": [{"position": [0.632884, -0.015352000000000032, -12.618986], "radius": 1e-05, "object_id": 602, "object_name": null, "object_category": null, "room_id": null, "room_name": null, "view_points": [[0.632884, -0.015352000000000032, -12.618986], [1.632884, -0.015352000000000032, -13.618986], [2.632884, -0.015352000000000032, -13.618986]]}], "start_room": null, "shortest_paths": null, "object_category": "tv_monitor", "sound_id": "test/tv_monitor.wav", "offset": "0", "duration": "18"}, {"episode_id": "145", "scene_id": "yqstnuAEVhm/yqstnuAEVhm.glb", "start_position": [-0.367116, -0.004817000000000071, -11.618986], "start_rotation": [0.0, 1.0, 0.0, 6.123233995736766e-17], "info": {"geodesic_distance": 8.0, "num_action": 

In [4]:
train_gz_path = "/home/Disk/sound-space/data/datasets/semantic_audionav/mp3d/v1/test/content/split_0.json.gz"   # TODO: 改成你的实际路径

fmt, data, episodes = read_train_json_gz(path)

print("Format:", fmt)
print("Total episodes:", len(episodes))

# 看看 episode 有哪些常见 key（取前 5 个统计一下）
from collections import Counter

key_counter = Counter()
for ep in episodes[:500]:  # 前500个够看了
    if isinstance(ep, dict):
        key_counter.update(ep.keys())

print("\nTop keys:")
for k, c in key_counter.most_common(20):
    print(f"  {k}: {c}")


NameError: name 'read_train_json_gz' is not defined